Kernel: PYTORCH 2.0.0 Python 3.10 CPU Kernel, ml.m5.xlarge

In [2]:
#install git lfs (required by hugging face model)
!apt update
!apt install gnupg -y
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | bash
!apt install -y git-lfs -y
!git lfs install

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]                
Get:3 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1129 kB]
Get:4 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [3221 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [3057 kB]33m
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]        m
Get:7 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [29.3 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]  
Get:9 http://archive.ubuntu.com/ubuntu focal/restricted amd64 Packages [33.4 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal/multiverse amd64 Packages [177 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal/main amd64 Packages [1275 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal/universe amd64 Packages [11.3 MB]
Get:13 http:

In [3]:
#Check pytorch successfully installed with CUDA enabled if using GPU instance
import torch
# torch.cuda.get_device_name(0)


In [4]:
#ffmpeg 4 is required for pytorch to process mp3
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt install -y ffmpeg


Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
Ign:2 https://packagecloud.io/github/git-lfs/ubuntu focal InRelease            
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease                         
Ign:4 http://ppa.launchpad.net/jonathonf/ffmpeg-4/ubuntu focal InRelease       
Hit:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease                 
Err:6 https://packagecloud.io/github/git-lfs/ubuntu focal Release      
  Certificate verification failed: The certificate is NOT trusted. The certificate issuer is unknown.  Could not handshake: Error in the certificate verification. [IP: 50.18.215.106 443]
Err:7 http://ppa.launchpad.net/jonathonf/ffmpeg-4/ubuntu focal Release 
  404  Not Found [IP: 185.125.190.80 80]
Hit:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Reading package lists... Done
W: https://packagecloud.io/github/git-lfs/ubuntu/dists/focal/InRelease: No system certificates available. Try installing ca-certificates.
W: https:/

In [5]:
#install transformers and other dependencies
!pip install --upgrade pip
!pip install datasets>=2.6.1 git+https://github.com/huggingface/transformers accelerate>=0.20.3 librosa evaluate>=0.3.0 jiwer gradio soundfile tqdm -U

  Using cached pip-23.3.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-uzlrstft
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
confection 0.0.4 requires pydantic!=1.8,!=1.8.1,<1.11.0,>=1.7.4, but you have pydantic 2.5.2 which is incompatible.
spacy 3.5.2 requires pydantic!=1.8,!=1.8.1,<1.11.0,>=1.7.4, but you have pydantic 2.5.2 which is incompatible.
spacy 3.5.2 requires typer<0.8.0,>=0.3.0, but you have typer 0.9.0 which is incompatible.
thinc 8.1.10 requires pydantic!=1.8,!=1.8.1,<1.11.0,>=1.7.4, but you have pydantic 2.5.2 which is incompatible.


In [6]:
#Download common voice dataset
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

train_dataset, validation_dataset, test_dataset = \
                load_dataset("mozilla-foundation/common_voice_11_0", "ga-IE", \
                             split=["train", "validation", "test"], use_auth_token=False)

print(train_dataset)


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/datasets/load.py:2088: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 532
})


In [7]:
#Assign corresponding splits into the dataset
common_voice["train"]=train_dataset
common_voice["validation"]=validation_dataset
common_voice["test"]=test_dataset
print(common_voice)


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 532
    })
    validation: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 513
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 514
    })
})


In [8]:
#Save the dataset to disk so that we can recover the dataset more easily.
common_voice.save_to_disk("ga-common-voice-original")

Saving the dataset (1/1 shards): 100%|██████████| 514/514 [00:02<00:00, 200.52 examples/s]


In [9]:
#Remove unneeded field for the training
from datasets import load_from_disk, DatasetDict
common_voice = load_from_disk("ga-common-voice-original")
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", 
                                            "gender", "locale", "path", "segment", "up_votes"])
common_voice

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 532
    })
    validation: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 513
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 514
    })
})

In [10]:
#Let's start from the pretrained small model
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-tiny", language="en", task="transcribe")
tokenizer = processor.tokenizer

In [11]:
#Verify that tokenizer works both ways
input_str = common_voice["train"][0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")


Input:                 Tá cuid agaibh ar bheagán Gaeilge agus cuid eile líofa ar fad
Decoded w/ special:    <|startoftranscript|><|en|><|transcribe|><|notimestamps|>Tá cuid agaibh ar bheagán Gaeilge agus cuid eile líofa ar fad<|endoftext|>
Decoded w/out special: Tá cuid agaibh ar bheagán Gaeilge agus cuid eile líofa ar fad
Are equal:             True


In [12]:
#Let's view a sample
print(common_voice["train"][0])

{'audio': {'path': 'common_voice_ga-IE_18182023.mp3', 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
       -3.11638723e-06, -2.60130837e-05,  1.24208600e-05]), 'sampling_rate': 48000}, 'sentence': 'Tá cuid agaibh ar bheagán Gaeilge agus cuid eile líofa ar fad'}


In [13]:
#Downsample the samples
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))
print(common_voice["train"][0])


{'audio': {'path': 'common_voice_ga-IE_18182023.mp3', 'array': array([-2.16840434e-19, -4.33680869e-19,  7.04731412e-19, ...,
       -1.94764580e-05,  4.15314425e-05,  3.63372092e-05]), 'sampling_rate': 16000}, 'sentence': 'Tá cuid agaibh ar bheagán Gaeilge agus cuid eile líofa ar fad'}


In [14]:
#Define the extraction function
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch


In [15]:
#Apply it to the whole dataset. Give it some time even all lines goes green.
# If the process failed, try delete the cache-xxxx arrow files under ga-common-voice-original/train or the ~/.cache and restart the kernel.
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=8)


In [16]:
#Save the processed dataset to disk
common_voice.save_to_disk("ga-common-voice-processed")

Saving the dataset (1/1 shards): 100%|██████████| 514/514 [00:07<00:00, 71.78 examples/s] 


In [17]:
common_voice

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 532
    })
    validation: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 513
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 514
    })
})

In [19]:
#Display sample test and save it for later usage
input_features = torch.FloatTensor([common_voice['test'][0]['input_features']])
expected = tokenizer.decode(common_voice['test'][0]['labels'], skip_special_tokens=False)
import pickle

with open('test.pickle', 'wb') as f:
    pickle.dump(common_voice['test'][0], f)

expected

'<|startoftranscript|><|en|><|transcribe|><|notimestamps|>Tá peann agamsa agus tá peann agatsa agus is linn féin iad<|endoftext|>'

In [20]:
#Test base transcription
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

# load model and processor
tprocessor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
tmodel = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
tmodel.config.forced_decoder_ids = None


# generate token ids
predicted_ids = tmodel.generate(input_features)
# decode token ids to text
transcription = tprocessor.batch_decode(predicted_ids, skip_special_tokens=False)

transcription

['<|startoftranscript|><|en|><|transcribe|><|notimestamps|> Topiaonagomsa Augustopiaonagomsa Augustopiaonagomsa August is Linfane Ed.<|endoftext|>']

In [ ]:
#View the default bucket location
import boto3
import sagemaker
import os
from sagemaker import get_execution_role
import os

sess = sagemaker.Session()
ROLE = get_execution_role()

BUCKET = sess.default_bucket() 
PREFIX = "whisper/data/ga-common-voice-processed"
s3uri = os.path.join("s3://", BUCKET, PREFIX)
s3uri

In [ ]:
#Use the aws s3 cli to upload the processed dataset. You could also choose to use the boto3 python sdk to do the upload.
!aws s3 cp --recursive ga-common-voice-processed {s3uri}